In [1]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime  
from nltk.tokenize import word_tokenize  
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
import numpy as np
import re
import mysql.connector

import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer

import tweepy
# importing relevant packages to receive Twitter streams
from tweepy import Stream
from tweepy.streaming import StreamListener
# credentials taken from the twitter API
import credentials
import json

from dateutil import parser

In [2]:
# importing relevant packages to receive Twitter streams
from tweepy import Stream
from tweepy.streaming import StreamListener
# credentials taken from the twitter API
import credentials




In [3]:
# credentials to access Twitter API
auth  = tweepy.OAuthHandler(credentials.API_KEY, \
                            credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN,  \
                      credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


In [4]:
# create function to strip emojis from the stream
def removeEmoj(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [5]:
# connecting the MYSQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="twitter",
    charset = 'utf8'
)

ProgrammingError: 1049 (42000): Unknown database 'twitter'

In [ ]:
# if db.is_connected():
#     '''
#     Check if this table exits. If not, then create a new one.
#     '''
#     mycursor = db.cursor()
#     mycursor.execute("""
#         SELECT COUNT(*)
#         FROM information_schema.tables
#         WHERE table_name = '{0}'
#         """.format("twitter"))
#     if mycursor.fetchone()[0] != 1:
#         mycursor.execute("CREATE TABLE (id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), user_created_at (VARCHAR), user_location VARCHAR(255))")
# # created_at DATETIME, text VARCHAR(255), user_craeted_at VARCHAR(255), user_location VARCHAR(255))"
                                                       
                                                    
#     db.commit()
#     mycursor.close()

In [ ]:

mycursor = db.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)
    

In [ ]:
mycursor.execute("SELECT * FROM twitter")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

In [ ]:
def store_data(id_str , created_at, text, user_created_at, user_location):
    db=mysql.connector.connect(host="localhost", user="root", passwd="password", db="twitter", charset="utf8")
    cursor = db.cursor()
    insert_query = "INSERT INTO twitter (id_str, created_at, text, user_created_at, user_location) VALUES (%s, %s, %s, %s, %s)"
    cursor.execute(insert_query, (id_str , created_at, text, user_created_at,user_location ))
    db.commit()
    cursor.close()
    db.close()
    return

In [ ]:


# # customising the stream listener to add extra conditions
# class MyListener(StreamListener):
 
#     def on_data(self, status):
#         #ignore retweeted info and just get original         
# #         if status.retweeted:
# #             return True
#         #extracting all info from the stream and also taking out emojis for DB
         
#         id_str = status.id_str
#         created_at = status.created_at
#         text = removeEmoj(status.text)
#         user_created_at = status.user.created_at
#         user_location = removeEmoj(status.user.location)
    
#     # Store all data in MySQL
#     if db.is_connected():
#             cursor = db.cursor()
#             sql = "INSERT INTO {} (id_str, created_at, text, user_created_at, user_location, user_description) VALUES (%s, %s, %s, %s, %s, %s )".format('twitter')
#             values = (id_str, created_at, text, user_created_at, user_location, \
#                 user_description)
#             cursor.execute(sql, values)
#             db.commit()
#             cursor.close()
    
            
#     # if the limit of tweets is reached it returns an error state         
#     def on_error(self, status):
#         if status_code == 420:
#             return False


In [ ]:


# customising the stream listener to add extra conditions
class MyListener(StreamListener):
 

            
            
            
    def on_data(self, data):
        
        try:
           # Decode the JSON from Twitter
            datajson = json.loads(data)
    
            
            
            
                
                            #grab the wanted data from the Tweet
            id_str = datajson['id_str']
            print(1)
            created_at =parser.parse(datajson['created_at'])
#             created_at = datajson['created_at']
            print(2)
            text = datajson['text']
            text = removeEmoj(text)
            
            print(3)
#             user_created_at = parser.parse(datajson['user_created_at']) 
            user_created_at = 100
            print("4- " + str(datajson))
#             user_location = datajson['user_location']
            user_location = 200
            print(5)

   

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(text))
            
            #insert the data into the MySQL database
            store_data(id_str , created_at, text, user_created_at, user_location)
            print(datajson)
        
        except Exception as e:
            print(e)      
            #print out a message to the screen that we have collected a tweet
#             print("Tweet collected at " + str(created_at))
            
            #insert the data into the MySQL database
            
    
#     # Store all data in MySQL
#         if db.is_connected():
#             cursor = db.cursor()
#             sql = "INSERT INTO {} (id_str, created_at, text, user_created_at, user_location, user_description) VALUES (%s, %s, %s, %s, %s, %s )".format('twitter')
#             values = (id_str, created_at, text, user_created_at, user_location, \
#                 user_description)
#             cursor.execute(sql, values)
#             db.commit()
#             cursor.close()
    
            
#     # if the limit of tweets is reached it returns an error state         
#         def on_error(self, status):
#             if status_code == 420:
#             return False


In [ ]:
MyListener = MyListener()
myStream = tweepy.Stream(auth = api.auth, listener = MyListener )
myStream.filter(languages=["en"], track = ['#coronavirus'])